## 4.3 가중치 초기화 함수(1)

In [1]:
from torch import nn


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(1, 2),
            nn.Sigmoid()
        )
        self.fc = nn.Linear(2, 1)
        self._init_weights()

    def _init_weights(self):
        nn.init.xavier_uniform_(self.layer[0].weight)  #제이비어 초기화(균등 분포)
        self.layer[0].bias.data.fill_(0.01)

        nn.init.xavier_uniform_(self.fc.weight)
        self.fc.bias.data.fill_(0.01)


model = Net()

## 4.4 가중치 초기화 함수(2)

In [3]:
from torch import nn


class Net(nn.Module):  # 신경망 Net을 정의
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(1, 2),  # 입력 뉴런 : 1 , 출력 뉴런 : 2
            nn.Sigmoid() #시그모이드 활성화 함수
        )
        self.fc = nn.Linear(2, 1)  # 두번째 Linear Layer. 2개의 입력을 받아 1개의 출력을 만듦.
        self.apply(self._init_weights) # 모델의 모든 레이어에 _init_weights 함수를 적용 하는 역할.

    def _init_weights(self, module): # 가중치 초기화 함수
        if isinstance(module, nn.Linear): # module 이 nn.Linear 인지 확인
            nn.init.xavier_uniform_(module.weight) # 제이비어 초기화(균등분포) 사용
            nn.init.constant_(module.bias, 0.01)
        print(f"Apply : {module}")


model = Net()

'''
출력결과는 초기화가 적용된 모든 레이와 모델이 출력된거임.
'''

Apply : Linear(in_features=1, out_features=2, bias=True)
Apply : Sigmoid()
Apply : Sequential(
  (0): Linear(in_features=1, out_features=2, bias=True)
  (1): Sigmoid()
)
Apply : Linear(in_features=2, out_features=1, bias=True)
Apply : Net(
  (layer): Sequential(
    (0): Linear(in_features=1, out_features=2, bias=True)
    (1): Sigmoid()
  )
  (fc): Linear(in_features=2, out_features=1, bias=True)
)


## 드롭 아웃

In [3]:
from torch import nn


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(10, 10)
        self.dropout = nn.Dropout(p=0.5)
        self.layer2 = nn.Linear(10, 10)

    def forward(self, x):
        x = self.layer1(x)
        x = self.dropout(x)
        x = self.layer2(x)
        return x

# __Data Augmentation__

## __텍스트 데이터__

### 삽입 및 삭제

In [2]:
!pip install numpy requests nlpaug transformers sacremoses nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 34.3 MB/s eta 0:00:00


#### 삽입

In [4]:
# CPU: 40초 넘게?
# GPU: 30초 정도?


import nlpaug.augmenter.word as naw


texts = [
    "Those who can imagine anything, can create the impossible.",
    "We can only see a short distance ahead, but we can see plenty there that needs to be done.",
    "If a machine is expected to be infallible, it cannot also be intelligent.",
]

aug = naw.ContextualWordEmbsAug(model_path="bert-base-uncased", action="insert") #ContextualWordEmbsAug(BERT 모델사용) , action : insert, substitute(대체)
augmented_texts = aug.augment(texts)

for text, augmented in zip(texts, augmented_texts):
    print(f"src : {text}") # 원래 거
    print(f"dst : {augmented}") # 적용 된 거
    print("------------------")

src : Those who can imagine anything, can create the impossible.
dst : certainly those few who can even imagine anything, can create the complete impossible.
------------------
src : We can only see a short distance ahead, but we can see plenty there that needs to be done.
dst : we can actually only see a short driving distance ahead, but we also can well see around plenty by there that needs to be hastily done.
------------------
src : If a machine is expected to be infallible, it cannot also be intelligent.
dst : even if a nuclear machine is now expected to be spontaneously infallible, because it cannot also be immediately intelligent.
------------------


#### 삭제

In [5]:
import nlpaug.augmenter.char as nac


texts = [
    "Those who can imagine anything, can create the impossible.",
    "We can only see a short distance ahead, but we can see plenty there that needs to be done.",
    "If a machine is expected to be infallible, it cannot also be intelligent.",
]

aug = nac.RandomCharAug(action="delete") # RandomCharAug 클래스 사용. action : insert, substitue, swap,delete 사용 가능
augmented_texts = aug.augment(texts)

for text, augmented in zip(texts, augmented_texts):
    print(f"src : {text}")
    print(f"dst : {augmented}")
    print("------------------")

src : Those who can imagine anything, can create the impossible.
dst : hoe who can agne anything, can rate the impsibe.
------------------
src : We can only see a short distance ahead, but we can see plenty there that needs to be done.
dst : We can oy see a hrt disce aed, but we can see plty there th needs to be de.
------------------
src : If a machine is expected to be infallible, it cannot also be intelligent.
dst : If a machine is expte to be ifllibe, it cano al be nelient.
------------------


#### 교체 및 대체

In [6]:
import nlpaug.augmenter.word as naw


texts = [
    "Those who can imagine anything, can create the impossible.",
    "We can only see a short distance ahead, but we can see plenty there that needs to be done.",
    "If a machine is expected to be infallible, it cannot also be intelligent.",
]

aug = naw.RandomWordAug(action="swap") # RandomWordAug 클래스를 사용. 근데 그럼 문맥이 어색해 질 수 있는 문제가 발생
augmented_texts = aug.augment(texts)

for text, augmented in zip(texts, augmented_texts):
    print(f"src : {text}")
    print(f"dst : {augmented}")
    print("------------------")

src : Those who can imagine anything, can create the impossible.
dst : Those who can anything imagine create, can the. impossible
------------------
src : We can only see a short distance ahead, but we can see plenty there that needs to be done.
dst : We can a only see short ahead distance, but can we plenty see there that needs to be done.
------------------
src : If a machine is expected to be infallible, it cannot also be intelligent.
dst : A if machine is expected to be, infallible cannot also it intelligent be.
------------------


In [8]:
import nlpaug.augmenter.word as naw
import nltk

# Download the required NLTK data
nltk.download('averaged_perceptron_tagger_eng')

texts = [
    "Those who can imagine anything, can create the impossible.",
    "We can only see a short distance ahead, but we can see plenty there that needs to be done.",
    "If a machine is expected to be infallible, it cannot also be intelligent.",
]

aug = naw.SynonymAug(aug_src='wordnet') #SynonymAug 크래스는 워드넷 데이터베이스나 의역 데이터베이스를 활용해 단어를 대체해 데이터를 증가. 동의어, 유의어로 변경
augmented_texts = aug.augment(texts)

for text, augmented in zip(texts, augmented_texts):
    print(f"src : {text}")
    print(f"dst : {augmented}")
    print("------------------")

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


src : Those who can imagine anything, can create the impossible.
dst : Those who terminate guess anything, can produce the impossible.
------------------
src : We can only see a short distance ahead, but we can see plenty there that needs to be done.
dst : We tin can just construe a little distance forward, but we keister see plenty there that needs to be done.
------------------
src : If a machine is expected to be infallible, it cannot also be intelligent.
dst : If a car is expected to exist infallible, it cannot also be reasoning.
------------------


In [9]:
import nlpaug.augmenter.word as naw


texts = [
    "Those who can imagine anything, can create the impossible.",
    "We can only see a short distance ahead, but we can see plenty there that needs to be done.",
    "If a machine is expected to be infallible, it cannot also be intelligent.",
]
reserved_tokens = [
    ["can", "can't", "cannot", "could"],
]

reserved_aug = naw.ReservedAug(reserved_tokens=reserved_tokens)
augmented_texts = reserved_aug.augment(texts)

for text, augmented in zip(texts, augmented_texts):
    print(f"src : {text}")
    print(f"dst : {augmented}")
    print("------------------")

src : Those who can imagine anything, can create the impossible.
dst : Those who can't imagine anything, could create the impossible.
------------------
src : We can only see a short distance ahead, but we can see plenty there that needs to be done.
dst : We could only see a short distance ahead, but we could see plenty there that needs to be done.
------------------
src : If a machine is expected to be infallible, it cannot also be intelligent.
dst : If a machine is expected to be infallible, it can't also be intelligent.
------------------


In [ ]:
import nlpaug.augmenter.word as naw


texts = [
    "Those who can imagine anything, can create the impossible.",
    "We can only see a short distance ahead, but we can see plenty there that needs to be done.",
    "If a machine is expected to be infallible, it cannot also be intelligent.",
]

back_translation = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de',
    to_model_name='facebook/wmt19-de-en'
)
augmented_texts = back_translation.augment(texts)

for text, augmented in zip(texts, augmented_texts):
    print(f"src : {text}")
    print(f"dst : {augmented}")
    print("------------------")

config.json:   0%|          | 0.00/825 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-en-de and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/825 [00:00<?, ?B/s]

...

vocab-src.json:   0%|          | 0.00/849k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/315k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

vocab-src.json:   0%|          | 0.00/849k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/315k [00:00<?, ?B/s]